In [1]:
# %pip install -q requests

In [2]:
import requests
import os

# MAAP API -- OGC API Features Proxy Test

## Querying ATL08 EPT using the Features API

We can use the Features Service for some basic querying of the EPT Store.

> OGC API Features provides API building blocks to create, modify and query features on the Web.

[Documentation: OGC API - Features](https://www.ogc.org/standards/ogcapi-features)

### Query By Bounding Box

Using EPT and the features service delivers a response **100x faster** over the conventional way of subsetting this data (search, download, read and subset data). See the test in [subset-atl08-files.ipynb](subset-atl08-files.ipynb).

In [3]:
# Format a request to the API
api_url_root = "http://localhost:5000/api/ogcapi-features"

# the wfs_pdal service does not yet support the root/landing page or service-desc resources

r = requests.get(f"{api_url_root}/conformance")
assert r.status_code == 200, f"conformance resource failed with status_code={r.status_code}"

r = requests.get(f"{api_url_root}/collections")
assert r.status_code == 200, f"collections resource failed with status_code={r.status_code}"

r = requests.get(f"{api_url_root}/collections/Global")
assert r.status_code == 200, f"collection Global resource failed with status_code={r.status_code}"

r = requests.get(f"{api_url_root}/collections/Global/queryables")
assert r.status_code == 200, f"collection Global queryables resource failed with status_code={r.status_code}"

r = requests.get(f"{api_url_root}/collections/Global/items")
assert r.status_code == 200, f"collection Global items resource failed with status_code={r.status_code}"

# no test for the /collections/{collection_id}/items/{item_id} endpoint because the features don't have IDs

# Make a request for a bounding box over Peru
payload = {
    "limit": 1000,
    "bbox": "-77,-26,300,-73,0,500"
}

r = requests.get(f"{api_url_root}/collections/Global/items", params=payload)
assert r.status_code == 200, f"query on Global collection failed with status_code={r.status_code}"
api_geojson = r.json()
assert len(r.json().get("features",[])) >= 1

# limit
r = requests.get(f"{api_url_root}/collections/Global/items", params={ "limit": 1 })
assert r.status_code == 200, f"collection Global items resource with limit failed with status_code={r.status_code}"
assert len(r.json().get("features",[])) == 1

### Query by Granule Id

In [4]:
granule_id = 'ATL08_20181014035224_02370107_003_01'
r = requests.get(f"{api_url_root}/collections/Global/items", params = { "origin": granule_id })
assert len(r.json().get("features",[])) >= 1

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

### Errors

In [ ]:
r = requests.get(f"{api_url_root}/invalid-path")
assert r.status_code == 404 , f"invalid resource path returned status_code={r.status_code}"

r = requests.get(f"{api_url_root}/collections/non-existent")
# The OGC API spec says a non-existent collection should return a 404, but pygeoapi returns 400, but the proxy is just
# passing it through anyway, so accept whichever one is returned
assert r.status_code == 400 or r.status_code == 404 , f"non-existent collection resource returned status_code={r.status_code}"

r = requests.get(f"{api_url_root}/collections/Global/items/some-id")
# This should return a 404, but it returns 422 for some reason. May be fixed in pygeoapi 0.10.1
assert r.status_code == 422 , f"non-existent item resource returned status_code={r.status_code}"

r = requests.get(f"{api_url_root}/collections/Global/items/1")
# This should return a 404, but it returns 500 for some reason. May be fixed in pygeoapi 0.10.1
assert r.status_code == 500 , f"non-existent item resource returned status_code={r.status_code}"

r = requests.get(f"{api_url_root}/collections/Global/items", params={ "bbox": "-77,-26,-73" })
assert r.status_code == 400, f"query on items with invalid bbox failed with status_code={r.status_code}"